In [201]:
from pararel.consistency import utils
import os
import pandas as pd
import requests
from collections import defaultdict
import numpy as np

## Investigating the granularity issues in ParaRel relations' options

In [202]:
#read data
relation = "P937"
options = set()

data = utils.read_jsonl_file(os.path.join("data/trex_lms_vocab", relation + ".jsonl"))
data_lama = utils.read_jsonl_file(os.path.join("data/lama_data/TREx", relation + ".jsonl"))

for dp in data:
    options.update({dp["obj_label"]})

In [203]:
#get wikidata ids from lama data
pd_data = pd.DataFrame(data)
pd_data_lama = pd.DataFrame(data_lama)

pd_merged_data = pd_data.merge(pd_data_lama[["uuid", "obj_uri", "sub_uri"]], on='uuid', how='left')
merged_data = pd_merged_data.to_dict('records')
pd_merged_data.head()

,sub_label,obj_label,uuid,obj_uri,sub_uri
0,Howard Florey,London,d515f9f9-9ad3-4493-a342-aad6d2d76e41,Q84,Q137106
1,Enki Bilal,Paris,dbec9b61-92e1-4501-902c-360dfc173b1b,Q90,Q333668
2,Pedro de Ribera,Madrid,e7b51701-97b6-4b36-ad10-be45313e0e8e,Q2807,Q3049047
3,Ramsay MacDonald,London,cc6138fe-37ee-4817-b431-e00a7fd54562,Q84,Q166646
4,George Albertus Cox,Ottawa,3c1a8329-ecce-436a-b34d-196788091b3e,Q1930,Q5536134


In [204]:
#get all countries
#includes country, sovereign state or constituent country of the United Kingdom
url = 'https://query.wikidata.org/sparql'
query = '''
SELECT ?item ?itemLabel 
WHERE 
{
  VALUES ?id {wd:Q3624078 wd:Q6256 wd:Q3336843}
  ?item wdt:P31 ?id .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?item ?itemLabel 
'''

r = requests.get(url, params = {'format': 'json', 'query': query})
all_countries_data = r.json()

all_countries = defaultdict()
for i in all_countries_data["results"]["bindings"]:
    all_countries[i["item"]["value"].split("/")[-1]] = i["itemLabel"]["value"]

In [205]:
#get all wikidata ids to lookup
to_query = set()
for i in merged_data:
    to_query.update({"wd:"+i["obj_uri"]})
to_query_str = " ".join([i for i in to_query])

In [206]:
#get city to coutry from wikidata

'https://query.wikidata.org/sparql'
q1 = "SELECT DISTINCT ?id ?country ?countryLabel WHERE { VALUES ?id {"   
q2 = "} ?id wdt:P17 ?country . SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\" . } }"
query = q1.strip() + to_query_str.strip() + q2.strip()
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

id_to_country = defaultdict(tuple)
for i in data["results"]["bindings"]:
    id_to_country[i["id"]["value"].split("/")[-1]] = (i["country"]["value"].split("/")[-1], i["countryLabel"]["value"])

In [207]:
#add coutry info if found
for i in merged_data:
    if id_to_country[i["obj_uri"]]:
        i["countryId"] = id_to_country[i["obj_uri"]][0]
        i["country"] = id_to_country[i["obj_uri"]][1]
    else:
        i["countryId"] = None

In [208]:
#count afected data points
count_country = 0
count_city_with_country = 0
uk_city = 0
for i in merged_data:
    if i["obj_uri"] in all_countries.keys():
        count_country+=1
    if i["obj_uri"] not in all_countries.keys() and i["countryId"] in country_answers:
        count_city_with_country+=1
    if i["countryId"] == "Q145":
        uk_city +=1

percent_affected = round(np.sum([count_country, count_city_with_country, uk_city,])/len(merged_data),3)
count_country, count_city_with_country, uk_city, len(merged_data), percent_affected

(10, 75, 162, 832, 0.297)